In [16]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import plotly.graph_objs as go

# Carregar o dataframe
file_path = '0001_Master_modificado.csv'  # Substitua pelo caminho correto
data = pd.read_csv(file_path)

# Converter a coluna 'Data_Hora_Medicao' para datetime
data['Data_Hora_Medicao'] = pd.to_datetime(data['Data_Hora_Medicao'])

# Filtrar os dados para o período de treinamento e de previsão
train_data = data[(data['Data_Hora_Medicao'] >= '2019-01-01') & (data['Data_Hora_Medicao'] <= '2023-12-31')]
test_data = data[(data['Data_Hora_Medicao'] >= '2024-01-01') & (data['Data_Hora_Medicao'] <= '2024-08-31')]

# Definir as colunas de entrada (features) e a coluna alvo (target)
X_train = train_data.drop(columns=['Data_Hora_Medicao', 'Cota_Adotada_86510000'])  # Usar todas as colunas menos a alvo e a de tempo
y_train = train_data['Cota_Adotada_86510000']  # Coluna alvo é Cota_Adotada_86510000

# Preparar os dados de teste
X_test = test_data.drop(columns=['Data_Hora_Medicao', 'Cota_Adotada_86510000'])
y_test = test_data['Cota_Adotada_86510000']

# Criar o modelo Random Forest
#model = RandomForestRegressor(n_estimators=500, min_samples_leaf=5, random_state=42)
model = RandomForestRegressor(n_estimators=1000, max_depth=15, min_samples_split=10, random_state=42)


# Treinar o modelo
model.fit(X_train, y_train)

# Fazer a previsão para o período de 01/01/2024 a 31/08/2024
predictions = model.predict(X_test)

# Converter de centímetros para metros
y_test_metros = y_test / 100  # Converter os valores reais para metros
predictions_metros = predictions / 100  # Converter as previsões para metros

# Avaliar o modelo
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

# Criar o gráfico interativo com Plotly, mantendo as casas decimais e com valores em metros
fig = go.Figure()

# Adicionar os valores reais ao gráfico (convertidos para metros)
fig.add_trace(go.Scatter(x=test_data['Data_Hora_Medicao'], y=y_test_metros, mode='lines', name='Valores Reais (metros)', line=dict(color='blue')))

# Adicionar as previsões ao gráfico (convertidos para metros)
fig.add_trace(go.Scatter(x=test_data['Data_Hora_Medicao'], y=predictions_metros, mode='lines', name='Previsões (metros)', line=dict(color='red')))

# Customizar o layout do gráfico
fig.update_layout(
    title="Previsão de Níveis do Rio para o Período de 01/01/2024 até 31/08/2024 (em metros)",
    xaxis_title="Data",
    yaxis_title="Nível do Rio (metros)",
    hovermode="x unified"
)

# Ajustar o hover para exibir duas casas decimais
fig.update_traces(hovertemplate='%{y:.2f} metros')

# Exibir o gráfico interativo
fig.show()


Mean Squared Error: 37.23990427336586


# Código com Random Forest e Cross-Validation:

In [17]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import plotly.graph_objs as go

# Carregar o dataframe
file_path = '0001_Master_modificado.csv'  # Substitua pelo caminho correto
data = pd.read_csv(file_path)

# Converter a coluna 'Data_Hora_Medicao' para datetime
data['Data_Hora_Medicao'] = pd.to_datetime(data['Data_Hora_Medicao'])

# Filtrar os dados para o período de treinamento e de previsão
train_data = data[(data['Data_Hora_Medicao'] >= '2019-01-01') & (data['Data_Hora_Medicao'] <= '2023-12-31')]
test_data = data[(data['Data_Hora_Medicao'] >= '2024-01-01') & (data['Data_Hora_Medicao'] <= '2024-08-31')]

# Definir as colunas de entrada (features) e a coluna alvo (target)
X_train = train_data.drop(columns=['Data_Hora_Medicao', 'Cota_Adotada_86510000'])  # Usar todas as colunas menos a alvo e a de tempo
y_train = train_data['Cota_Adotada_86510000']  # Coluna alvo é Cota_Adotada_86510000

# Preparar os dados de teste
X_test = test_data.drop(columns=['Data_Hora_Medicao', 'Cota_Adotada_86510000'])
y_test = test_data['Cota_Adotada_86510000']

# Criar o modelo Random Forest com Cross-Validation
model = RandomForestRegressor(n_estimators=500, min_samples_leaf=5, random_state=42)

# Validação cruzada com 5 divisões
scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

# Média do MSE com validação cruzada
mean_mse = -scores.mean()
print(f'Média do MSE com validação cruzada: {mean_mse}')

# Treinar o modelo usando todos os dados de treino
model.fit(X_train, y_train)

# Fazer a previsão para o período de 01/01/2024 a 31/08/2024
predictions = model.predict(X_test)

# Converter de centímetros para metros
y_test_metros = y_test / 100  # Converter os valores reais para metros
predictions_metros = predictions / 100  # Converter as previsões para metros

# Avaliar o modelo
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

# Criar o gráfico interativo com Plotly, mantendo as casas decimais e com valores em metros
fig = go.Figure()

# Adicionar os valores reais ao gráfico (convertidos para metros)
fig.add_trace(go.Scatter(x=test_data['Data_Hora_Medicao'], y=y_test_metros, mode='lines', name='Valores Reais (metros)', line=dict(color='blue')))

# Adicionar as previsões ao gráfico (convertidos para metros)
fig.add_trace(go.Scatter(x=test_data['Data_Hora_Medicao'], y=predictions_metros, mode='lines', name='Previsões (metros)', line=dict(color='red')))

# Customizar o layout do gráfico
fig.update_layout(
    title="Previsão de Níveis do Rio para o Período de 01/01/2024 até 31/08/2024 (em metros)",
    xaxis_title="Data",
    yaxis_title="Nível do Rio (metros)",
    hovermode="x unified"
)

# Ajustar o hover para exibir duas casas decimais
fig.update_traces(hovertemplate='%{y:.2f} metros')

# Exibir o gráfico interativo
fig.show()


Média do MSE com validação cruzada: 1596.589971960319
Mean Squared Error: 31.79777069390319


# Código com Gradient Boosting:

In [18]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import plotly.graph_objs as go

# Carregar o dataframe
file_path = 'caminho/para/seu/arquivo/0001_Master_modificado.csv'  # Substitua pelo caminho correto
data = pd.read_csv(file_path)

# Converter a coluna 'Data_Hora_Medicao' para datetime
data['Data_Hora_Medicao'] = pd.to_datetime(data['Data_Hora_Medicao'])

# Filtrar os dados para o período de treinamento e de previsão
train_data = data[(data['Data_Hora_Medicao'] >= '2019-01-01') & (data['Data_Hora_Medicao'] <= '2023-12-31')]
test_data = data[(data['Data_Hora_Medicao'] >= '2024-01-01') & (data['Data_Hora_Medicao'] <= '2024-08-31')]

# Definir as colunas de entrada (features) e a coluna alvo (target)
X_train = train_data.drop(columns=['Data_Hora_Medicao', 'Cota_Adotada_86510000'])  # Usar todas as colunas menos a alvo e a de tempo
y_train = train_data['Cota_Adotada_86510000']  # Coluna alvo é Cota_Adotada_86510000

# Preparar os dados de teste
X_test = test_data.drop(columns=['Data_Hora_Medicao', 'Cota_Adotada_86510000'])
y_test = test_data['Cota_Adotada_86510000']

# Criar o modelo Gradient Boosting
model = GradientBoostingRegressor(n_estimators=500, learning_rate=0.05, max_depth=10, random_state=42)

# Treinar o modelo
model.fit(X_train, y_train)

# Fazer a previsão para o período de 01/01/2024 a 31/08/2024
predictions = model.predict(X_test)

# Converter de centímetros para metros
y_test_metros = y_test / 100  # Converter os valores reais para metros
predictions_metros = predictions / 100  # Converter as previsões para metros

# Avaliar o modelo
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

# Criar o gráfico interativo com Plotly, mantendo as casas decimais e com valores em metros
fig = go.Figure()

# Adicionar os valores reais ao gráfico (convertidos para metros)
fig.add_trace(go.Scatter(x=test_data['Data_Hora_Medicao'], y=y_test_metros, mode='lines', name='Valores Reais (metros)', line=dict(color='blue')))

# Adicionar as previsões ao gráfico (convertidos para metros)
fig.add_trace(go.Scatter(x=test_data['Data_Hora_Medicao'], y=predictions_metros, mode='lines', name='Previsões (metros)', line=dict(color='red')))

# Customizar o layout do gráfico
fig.update_layout(
    title="Previsão de Níveis do Rio para o Período de 01/01/2024 até 31/08/2024 (em metros)",
    xaxis_title="Data",
    yaxis_title="Nível do Rio (metros)",
    hovermode="x unified"
)

# Ajustar o hover para exibir duas casas decimais
fig.update_traces(hovertemplate='%{y:.2f} metros')

# Exibir o gráfico interativo
fig.show()


FileNotFoundError: [Errno 2] No such file or directory: 'caminho/para/seu/arquivo/0001_Master_modificado.csv'